In [7]:
%reload_ext autoreload
%autoreload 2

In [4]:
# Modulos da propria aplicacao
from fonte_dados.fabrica import FabricaFonteDados, NERDS_VIAJANTES, WIKIPEDIA
from limpeza import limpeza_posts, limpeza_texto
from repository import nerds_viajantes, wikipedia, mongo_utils
from treinamento import dicionario, treinamento_lda
from treinamento.treinamento_lda import TreinamentoLda

from repository.mongo_utils import get_pages_content_collection
from repository.wikipedia import WikipediaRepo
from similarity.similarity import SimilarityCalculator

import numpy as np
import pandas as pd
import time

wikipedia_repo = WikipediaRepo(collection=get_pages_content_collection())

In [8]:
fabrica = FabricaFonteDados()

In [7]:
origem = NERDS_VIAJANTES
destino = WIKIPEDIA

## Trabalho com base de treinamento

In [9]:
fonte_dados_origem = fabrica.get_fonte_dados(origem)
fonte_dados_origem.carregar_dados()
documentos_origem = fonte_dados_origem.get_tokens()

### Ajuste de modelo para posts de base de treinamento (origem)

In [14]:
# Ajusta modelo LDA para os documentos de origem. O resultado gerado contem o dicionario, o corpus de dados e o modelo gerado
treinamento_lda = TreinamentoLda(num_topics=100, passes=2)
resultado_lda = treinamento_lda.ajustar_modelo(documentos_origem)

Ajustando modelo com 100 topicos e 2 passes


### Definicao de probabilidades de topicos para documentos de origem

In [16]:
probabilidades_topicos_origem = [treinamento_lda.calcular_probabilidades_documento(dnv, resultado_lda) for dnv in documentos_origem]
np.array(probabilidades_topicos_origem).shape

(378, 100)

### Teste com classe

## Trabalho com base de comparaçã (destino)

In [17]:
fonte_dados_destino = fabrica.get_fonte_dados(destino)
fonte_dados_destino.carregar_dados()
documentos_destino = fonte_dados_destino.get_tokens()

In [18]:
fonte_dados_destino.get_dataframe().head()

,pageid,title,country,text_clean,tokens,n_tokens
0,22911,Geografia da Argentina,Argentina,A Geografia da Argentina é um domínio de estud...,"[geograf, argentin, domíni, estud, conhec, sob...",1163
1,1996157,Banco Namuncurá,Argentina,Banco Namuncurá (Banco Burdwood) e uma platafo...,"[banc, namuncur, banc, burdwood, plataform, su...",37
2,1393635,Campo de gelo do sul da Patagónia,Argentina,O Campo de gelo do sul da Patagónia (em castel...,"[camp, gel, sul, patagón, castelh, hiel, conti...",181
3,3425259,Cerro de la Gloria,Argentina,\nPico da Glória é um morro (cerro) próximo a...,"[pic, glór, morr, cerr, próx, cidad, argentin,...",203
4,1753443,Chaco Austral,Argentina,Chaco Austral é uma das três principais divisõ...,"[chac, austr, trê, princip, divis, chac, porç,...",62


### Definicao de topicos para documentos de destino

- Quantidade de documentos: 2252
- Indice do maior documento: 123
- Tamanho do maior documento: 10018 tokens
- Tempo gasto para definir tópicos de maior documento: 0.02167201042175293 segundos
- Tempo estimado para todos os documentos (com base no tempo do maior): 18.83 segundos
    * Na verdade este valor variou muito, não é fixo, mas a maioria das vezes ficou em torno de 20 segundos

In [16]:
# TODO: Remover
def avaliar_tempos_definicao_topicos_wikipedia():
    print(f'Quantidade de documentos: {len(documentos_wikipedia)}')

    tamanhos = [len(d) for d in documentos_wikipedia]
    indice_maior_documento = np.argmax(tamanhos)
    print(f'Indice do maior documento: {indice_maior_documento}')
    print(f'Tamanho do maior documento: {tamanhos[indice_maior_documento]} tokens')

    maior_documento = documentos_wikipedia[indice_maior_documento]
    start_time = time.time()
    treinamento_lda.calcular_probabilidades_documento(maior_documento, lda_nerds_viajantes)
    tempo_gasto_maior_documento = time.time() - start_time
    print(f"Tempo gasto na definicao dos topicos do maior documento: {tempo_gasto_maior_documento:.2f} segundos")
    print(f'Tempo estimado para todos documentos: {len(documentos_wikipedia) * tempo_gasto_maior_documento:.2f} segundos')

# avaliar_tempos_definicao_topicos_wikipedia()

### Calculo de probabilidade de topicos para documentos de destino

In [21]:
start_time = time.time()
probabilidades_topicos_destino = [
    treinamento_lda.calcular_probabilidades_documento(dw, resultado_lda) for dw in documentos_destino]
elapsed_time = time.time() - start_time
print(f'Tempo total gasto na definicao dos topicos: {elapsed_time:.2f} segundos')

Tempo total gasto na definicao dos topicos: 6.57 segundos


## Calculo de semelhanca entre documentos de treinamento e comparação

In [24]:
print(f'Quantidade de documentos de origem: {len(probabilidades_topicos_origem)}')
print(f'Tamanho do array de probabilidades de documentos de origem: {len(probabilidades_topicos_origem[0])}')
print(f'Quantidade de documentos de destino: {len(probabilidades_topicos_destino)}')
print(f'Tamanho do array de probabilidades de documentos de destino: {len(probabilidades_topicos_destino[0])}')

Quantidade de documentos de origem: 378
Tamanho do array de probabilidades de documentos de origem: 100
Quantidade de documentos de destino: 2252
Tamanho do array de probabilidades de documentos de destino: 100


### Problema encontrado

Ao calcular as probabilidades que determinados documentos estejam em determinados tópicos vimos que poucos tópicos determinam um documento, independente de usarmos 13 ou 100 tópicos no modelo. Para calcular a matrix esparsa das similaridades eu acabei tendo que preencher um array com 0 em quase todas as posições para que não desse erro.
Devemos estudar para ver se o modelo LDA é realmente a melhor abordagem neste caso ou se não temos muitos poucos documentos.

Sem preencher com 0's os tópicos que não contribuiam para cada documento o que acontecia era que cada array de probabilidade por tópico de um documento tinha um tamanho diferente, o que dava erro ao criar a matrix esparsa, que espera que todos tenham a mesma quantidade de colunas, que no caso é a quantidade de tópicos.

UPDATE: Este documento não mais representa um problema pois ao usar o método calcular_probabilidades_documento este já retorno exatamente a quantidade de tópicos utilizada no treinamento.

In [25]:
similarity_calculator = SimilarityCalculator(probabilidades_topicos_destino)

### Calcula as documentos de comparação mais parecidos com os documentos de treinamento

In [26]:
destinos_mais_parecidos = [similarity_calculator.get_most_similar_documents(pto) for pto in probabilidades_topicos_origem]

## Análise específica - Nervs Viajantes versus Wikipedia

### Uma análise de post específico (bariloche-cerro-tronador-e-cascada-los-alerces)

Post escolhido do Nerds viajantes para analisar:

- id: 1356
- indice no array: 49
- name: bariloche-cerro-tronador-e-cascada-los-alerces

In [21]:
indice_post_nv = 49
posts.iloc[49][['name', 'title']]

name       bariloche-cerro-tronador-e-cascada-los-alerces
title    Bariloche - Cerro Tronador e Cascada los Alerces
Name: 49, dtype: object

In [30]:
bow_49 = lda_nerds_viajantes.corpus[49]
topics = lda_nerds_viajantes.modelo_lda.get_document_topics(bow_49)

"""
Topicos identificados para documento 49: 
[(6, 0.23247316), (91, 0.7655891)]
"""
# topics

# Visualizando topico 91, que contribui em 77%
lda_nerds_viajantes.modelo_lda.show_topic(topicid=91, topn=20)

[('dia', 0.014348166),
 ('visit', 0.011912983),
 ('lag', 0.009569725),
 ('algum', 0.009334474),
 ('pass', 0.009285997),
 ('temp', 0.008738895),
 ('fic', 0.007909209),
 ('estr', 0.007864787),
 ('par', 0.0073830336),
 ('ond', 0.007332809),
 ('parqu', 0.0067055314),
 ('cheg', 0.0066545513),
 ('cidad', 0.006438682),
 ('caminh', 0.0063042664),
 ('pod', 0.0061235996),
 ('fot', 0.0060603707),
 ('faz', 0.0060003907),
 ('pont', 0.005456806),
 ('vist', 0.0053889276),
 ('carr', 0.0053370255)]

In [32]:
from collections import Counter
print(f'Tamanho do documento 49: {len(documentos[49])}')
print(Counter(documentos[49]))

Tamanho do documento 49: 505
Counter({'cerr': 16, 'tron': 11, 'volt': 9, 'mão': 9, 'casc': 7, 'pont': 7, 'par': 7, 'lag': 7, 'caminh': 7, 'únic': 7, 'parqu': 6, 'carr': 6, 'estr': 6, 'tir': 6, 'fot': 6, 'chuv': 6, 'temp': 5, 'visit': 5, 'fic': 5, 'peg': 5, 'pouc': 5, 'segu': 5, 'algum': 5, 'hor': 5, 'ida': 5, 'dur': 4, 'argentin': 4, 'lo': 4, 'alerc': 4, 'rio': 4, 'bariloch': 4, 'permit': 4, 'bas': 4, 'pod': 4, 'aproximad': 4, 'nacion': 3, 'pert': 3, 'tant': 3, 'direç': 3, 'faz': 3, 'mascard': 3, 'direit': 3, 'part': 3, 'estreit': 3, 'guarit': 3, 'entr': 3, 'banh': 3, 'mans': 3, 'vai': 3, 'cheg': 3, 'mir': 3, 'cor': 3, 'impression': 3, 'lad': 3, 'dupl': 3, 'conhec': 2, 'bel': 2, 'nahuel': 2, 'huap': 2, 'mont': 2, 'negr': 2, 'com': 2, 'metr': 2, 'quant': 2, 'local': 2, 'saím': 2, 'tom': 2, 'hotel': 2, 'dirig': 2, 'vir': 2, 'bem': 2, 'conserv': 2, 'quilômetr': 2, 'pes': 2, 'estrang': 2, 'adiant': 2, 'atr': 2, 'sobr': 2, 'sequ': 2, 'prim': 2, 'interess': 2, 'estacion': 2, 'qued': 2, 'boni

In [28]:
paginas_wikipedia_mais_parecidas = wikipedia_mais_parecidos[indice_post_nv]
print(f'Wikipedia mais parecido: {pages_df.iloc[paginas_wikipedia_mais_parecidas, [1, 2]]}')

Wikipedia mais parecido:                                               title         country
1119                                    Rio Awatere   Nova Zelandia
2185                                     Rio Innoko  Estados Unidos
1109                                      Rio Arrow   Nova Zelandia
1194                                 Albion (Texas)  Estados Unidos
1258                                 Boston (Texas)  Estados Unidos
1789  Parque Nacional e Reserva Gates of the Arctic  Estados Unidos
1124                                    Rio Barrier   Nova Zelandia
894                   Lista de atrações de Auckland   Nova Zelandia
2188                                    Rio Nowitna  Estados Unidos
1138                                   Rio Turakina   Nova Zelandia


In [37]:
print(f'Tamanho do documento Rio Awetere (1119): {len(documentos_wikipedia[1119])}')
corpus_rio_awetere = lda_nerds_viajantes.dicionario.doc2bow(documentos_wikipedia[1119])
topics = lda_nerds_viajantes.modelo_lda.get_document_topics(corpus_rio_awetere)
"""
Topicos que mais contribuem para documento 1119:
Veja que o 91 eh o que mais contribui, dai a semelhanca com o post da cascada los alerces
[(6, 0.21784398), (53, 0.03848416), (87, 0.09624962), (91, 0.61537075)]
"""
print(topics)

Tamanho do documento Rio Awetere (1119): 35


[(6, 0.21784398), (53, 0.03848416), (87, 0.09624962), (91, 0.61537075)]

### Trabalho futuro

- Considerar remocao de palavras que estao contribuindo muito para formacao de determinados topicos e poderiam ser inuteis.
    * A palavra "dia", por exemplo, está aparecendo muito e parece pouco contribuir para a semelhança entre os documentos
- Considerar a possibilidade de traduzir algumas palavras que aparecem muito, como cascada, por exemplo

**Comentários sobre o resultado e trabalho futuro**

- Todos os posts que vem como os mais parecidos são relacionados a rios
- Os rios dos resultados são distribuídos entre os países Estados Unidos e Nova Zelândia mas curiosamente não Argentina
    * Provavelmente eu não baixei os posts de rios da Argentina

**Trabalho futuro**

- Analisar qual tópico mais aparece para este post
    * Analisar também do mais parecido na wikipedia
- Analisar palavras que mais contribuem para os tópicos em questão

### Análise com post específico

- circuito-w-las-torres-del-paine
    * id: 8218, indice: 256
- el-chalten-trilha-para-laguna-torre
    * id: 9829, indice: 290

In [194]:
posts.reset_index(inplace=True)
posts[posts['name'].str.contains('torre')]

,index,id,date,content,title,name,modified,comment_count,n_characters
251,369,8092,2014-03-17 09:00:21,Há muito tempo eu sonhava e planejava visitar ...,Circuito W - A Chegada ao Parque Torres del Paine,circuito-w-chegada-parque-torres-del-paine,2014-03-17 21:28:42,23,20206
256,374,8218,2014-03-24 09:00:37,"<span style=""line-height: 1.5em;"">O segundo di...",Circuito W - Las Torres del Paine,circuito-w-las-torres-del-paine,2014-09-03 07:58:53,25,31569
290,421,9829,2014-09-03 09:00:11,"Depois de fazermos o <a href=""http://www.nerds...",El Chaltén - Trilha para Laguna Torre,el-chalten-trilha-para-laguna-torre,2014-10-21 22:23:34,21,23382
365,893,14612,2017-06-19 18:39:35,"Uma das trilhas que fizemos em <a href=""http:/...",Vídeo - Episódio Laguna Torre,video-episodio-laguna-torre,2017-06-19 18:39:35,2,3803


In [202]:
indice_post_nv = 257
posts.iloc[indice_post_nv][['name', 'title']]

name       puerto-natales-pizzaria-mesita-grande
title    Puerto Natales - Pizzaria Mesita Grande
Name: 257, dtype: object

In [203]:
paginas_wikipedia_mais_parecidas = wikipedia_mais_parecidos[indice_post_nv]
print(f'Wikipedia mais parecido: {pages_df.iloc[paginas_wikipedia_mais_parecidas, [1, 2]]}')

Wikipedia mais parecido:                                                   title         country
541                              Junção tripla do Chile           Chile
830                                               Azapa           Chile
333                                            Anillaco       Argentina
231                               Frente País Solidário       Argentina
1360                                         Lowcountry  Estados Unidos
1390  Lista dos navios dos Estados Unidos afundados ...  Estados Unidos
793                               Tamarugal (província)           Chile
553                                     Alto del Carmen           Chile
544                                      Quinta Vergara           Chile
765                               Norte Grande do Chile           Chile


### Análise de tópicos

In [215]:
documento = documentos.values[indice_post_nv]
bow = [lda_nerds_viajantes.dicionario.doc2bow(documento)]

In [218]:
topics = lda_nerds_viajantes.modelo_lda.get_document_topics(bow)

In [219]:
"""
Resultado: Ha praticamente 100% de chance de o documento ser definido pelo topico 20
"""
for topic in topics:
    print(topic)

[(20, 0.99302703)]


In [224]:
lda_nerds_viajantes.modelo_lda.print_topic(20)

'0.014*"restaurant" + 0.010*"local" + 0.008*"visit" + 0.008*"conhec" + 0.007*"com" + 0.007*"maranh" + 0.007*"praç" + 0.007*"fot" + 0.007*"algum" + 0.006*"cidad"'